# Limpeza dos Dados

## Setup

In [1]:
import numpy as np 
import pandas as pd
from matplotlib import rcParams
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px
from shutil import copy2

# Matplotlib options
rcParams['figure.figsize'] = (10, 8)
rcParams['axes.grid'] = False
plt.style.use('seaborn-deep')

# Pandas setup
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000) 
pd.set_option('display.max_colwidth', -1) 

In [30]:
# Read file
path = "./Entities/"
name = "reator"
filename = name + ".csv"
df = pd.read_csv(path + filename, encoding='utf8')

# Transformations
df["DATA"] =  pd.to_datetime(df["DATA"], format="%Y-%m-%d")
# df.drop(columns=["ENTIDADE"], inplace=True)

# Select max value in month interval
period = df.DATA.dt.to_period("M")
tidy = df.groupby(period).mean()
tidy.index = tidy.index.rename("DATE")
# tidy.drop(columns="DATA", inplace=True)
tidy.head()
df = tidy

# Remover colunas nulas
aux = pd.DataFrame(df.isnull().sum() / df.shape[0], columns=["NULL_COUNT"])
threshold = 1
features = aux[aux["NULL_COUNT"] < threshold].index.tolist()
filtered = df[features]
filtered["ENTIDADE"] = name.upper()

print(
    f"{df.shape[1] - df[features].shape[1]} colunas removidas:\n"
    f"{set(df.columns) - set(features)}"
)

threshold = 60
percentage = (1 - filtered.isnull().sum() / filtered.shape[0]) * 100
percentage = pd.Series(np.around(percentage.values, 2), index=percentage.index)

### Ajustar dados com unidades diferentes
# Ajustar colunas do ano de 2015 cujos valores estão em µg para mg (padrão)
# resetted = filtered.reset_index()
# start_date = '2015-01'
# end_date = '2015-12'
# mask = (resetted["DATE"] >= start_date) & (resetted["DATE"] <= end_date)
# columns_to_fix = [
#     "ALUMÍNIO", "ARSÊNIO", "BÁRIO", "BORO", 
#     "CÁDMIO", "CHUMBO", "CIANETO", "COBALTO", 
#     "COBRE", "CROMO", "ESTANHO", "FERRO", 
#     "LÍTIO", "MANGANÊS", "MOLIBDÊNIO", "NÍQUEL"
# ]
# section = resetted[mask]

# for column in columns_to_fix:
#     section[column] = section[column].map(lambda x: x * 0.001)

# section_rows = section.index.tolist()
# filtered_dropped_volumes = resetted.drop(section_rows)
# filtered_merge = pd.concat([filtered_dropped_volumes, section])\
#     .sort_values("DATE")\
#     .set_index("DATE")
# df_fixed = filtered_merge[sorted(percentage.sort_values(ascending=False).index)]

filtered = filtered[sorted(filtered.columns)]

path = "./Tidy_Entities/"
filtered.to_csv(path + filename, encoding='utf8')

37 colunas removidas:
{'ARSÊNIO', 'ESTANHO', 'PH', 'COBALTO', 'NITROGÊNIO_KJELDAHL', 'COR_VERDADEIRA', 'LÍTIO', 'COMPOSTOS', 'COBRE', 'MANGANÊS', 'BÁRIO', 'PRATA', 'FERRO', 'MOLIBDÊNIO', 'ALUMÍNIO', 'PORCENTAGEM_SÓLIDOS', 'FENOL', 'SÓLIDOS', 'SURFACTANTES', 'NITROGÊNIO_AMONIACAL', 'NITROGÊNIO', 'FÓSFORO', 'ÍNDICE_DE_FENÓIS', 'NÍQUEL', 'BORO', 'CHUMBO', 'MERCÚRIO', 'DQO', 'ÓLEOS', 'CROMO_HEXAVALENTE', 'CÁDMIO', 'COLIFORMES_TERMOTOLERANTES', 'FLUORETO', 'DBO', 'CROMO', 'CIANETO', 'UMIDADE'}


In [94]:
print(
    f"\033[31mLinhas: {df.shape[0]}\n"
    f"Colunas: {df.shape[1]}\n\n"
    f"\033[32m-> Features: \n{df.columns.tolist()}\n\n"
    f"\033[34m-> Valores ÚNICOS: \n{df.nunique()}\n\n"
    f"\033[36m-> Valores FALTANTES: \n{df.isnull().sum()}\033[0"
)

Linhas: 88
Colunas: 39

-> Features: 
['SURFACTANTES', 'PH', 'FÓSFORO', 'ALUMÍNIO', 'ARSÊNIO', 'BÁRIO', 'BORO', 'CÁDMIO', 'CHUMBO', 'CIANETO', 'COBALTO', 'COBRE', 'CROMO', 'CROMO_HEXAVALENTE', 'DBO', 'DQO', 'ESTANHO', 'FERRO', 'FLUORETO', 'LÍTIO', 'MANGANÊS', 'MERCÚRIO', 'MOLIBDÊNIO', 'NÍQUEL', 'NITROGÊNIO_AMONIACAL', 'NITROGÊNIO_KJELDAHL', 'COLIFORMES_TERMOTOLERANTES', 'PRATA', 'ÍNDICE_DE_FENÓIS', 'NITROGÊNIO', 'FENOL', 'ALCALINIDADE', 'UMIDADE', 'PORCENTAGEM_SÓLIDOS', 'COMPOSTOS', 'COR_VERDADEIRA', 'SÓLIDOS', 'ÓLEOS', 'ACIDEZ']

-> Valores ÚNICOS: 
SURFACTANTES                  43
PH                            21
FÓSFORO                       71
ALUMÍNIO                      28
ARSÊNIO                       5 
BÁRIO                         22
BORO                          14
CÁDMIO                        3 
CHUMBO                        5 
CIANETO                       5 
COBALTO                       6 
COBRE                         13
CROMO                         8 
CROMO_HEXAVALE